<a href="https://colab.research.google.com/github/MahmoudRH/HemoVision/blob/Models/CBC_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
!pip install -r yolov5/requirements.txt

In [ ]:
import torch
from IPython.display import Image  
import os 
import random
import shutil

import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!git clone https://github.com/MahmudulAlam/Complete-Blood-Cell-Count-Dataset.git

In [ ]:
# to get the data from XML Annotation
def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()
    info_dict = {}
    info_dict['bboxes'] = []
    for elem in root:
        if elem.tag == "filename":
            info_dict['filename'] = elem.text
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))
            
            info_dict['image_size'] = tuple(image_size)

        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                    
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)            
            info_dict['bboxes'].append(bbox)
    print(info_dict)     
    return info_dict

In [ ]:
# Maping names with IDs
class_name_to_id_mapping = {"RBC": 0,
                           "WBC": 1,
                           "Platelets": 2
                           }

def convert_to_yolov5(info_dict,file):
    print_buffer = []
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
        b_center_x = (b["xmin"] + b["xmax"]) / 2 
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])
        
        image_w, image_h, image_c = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    save_file_name = os.path.join(f"/content/Complete-Blood-Cell-Count-Dataset/{file}/Annotations", info_dict["filename"].replace("jpg", "txt"))
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [ ]:
# Get the annotations
for i in ['Training','Validation','Testing']:
  annotations = [os.path.join(f"/content/Complete-Blood-Cell-Count-Dataset/{i}/Annotations", x) for x in os.listdir(f"/content/Complete-Blood-Cell-Count-Dataset/{i}/Annotations") if x[-3:] == "xml"]
  annotations.sort()
  for ann in tqdm(annotations):
      info_dict = extract_info_from_xml(ann)
      convert_to_yolov5(info_dict,i)
  annotations = [os.path.join(f"/content/Complete-Blood-Cell-Count-Dataset/{i}/Annotations", x) for x in os.listdir(f"/content/Complete-Blood-Cell-Count-Dataset/{i}/Annotations") if x[-3:] == "txt"]

In [ ]:
train_images = [os.path.join('/content/Complete-Blood-Cell-Count-Dataset/Training/Images', x) for x in os.listdir('/content/Complete-Blood-Cell-Count-Dataset/Training/Images')]
train_annotations = [os.path.join('/content/Complete-Blood-Cell-Count-Dataset/Training/Annotations', x) for x in os.listdir('/content/Complete-Blood-Cell-Count-Dataset/Training/Annotations') if x[-3:] == "txt"]

test_images = [os.path.join('/content/Complete-Blood-Cell-Count-Dataset/Testing/Images', x) for x in os.listdir('/content/Complete-Blood-Cell-Count-Dataset/Testing/Images')]
test_annotations = [os.path.join('/content/Complete-Blood-Cell-Count-Dataset/Testing/Annotations', x) for x in os.listdir('/content/Complete-Blood-Cell-Count-Dataset/Testing/Annotations') if x[-3:] == "txt"]

val_images = [os.path.join('/content/Complete-Blood-Cell-Count-Dataset/Validation/Images', x) for x in os.listdir('/content/Complete-Blood-Cell-Count-Dataset/Validation/Images')]
val_annotations = [os.path.join('/content/Complete-Blood-Cell-Count-Dataset/Validation/Annotations', x) for x in os.listdir('/content/Complete-Blood-Cell-Count-Dataset/Validation/Annotations') if x[-3:] == "txt"]


train_images.sort()
train_annotations.sort()

test_images.sort()
test_annotations.sort()

val_images.sort()
val_annotations.sort()


In [ ]:
import os 
os.mkdir("/content/Complete-Blood-Cell-Count-Dataset/new_dataset")
os.mkdir("/content/Complete-Blood-Cell-Count-Dataset/new_dataset/images")
os.mkdir("/content/Complete-Blood-Cell-Count-Dataset/new_dataset/labels")
os.mkdir("/content/Complete-Blood-Cell-Count-Dataset/new_dataset/images/train")
os.mkdir("/content/Complete-Blood-Cell-Count-Dataset/new_dataset/images/val")
os.mkdir("/content/Complete-Blood-Cell-Count-Dataset/new_dataset/images/test")
os.mkdir("/content/Complete-Blood-Cell-Count-Dataset/new_dataset/labels/train")
os.mkdir("/content/Complete-Blood-Cell-Count-Dataset/new_dataset/labels/val")
os.mkdir("/content/Complete-Blood-Cell-Count-Dataset/new_dataset/labels/test")
#Utility function to move images 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

# Move the splits into their folders
move_files_to_folder(train_images, '/content/Complete-Blood-Cell-Count-Dataset/new_dataset/images/train/')
move_files_to_folder(val_images, '/content/Complete-Blood-Cell-Count-Dataset/new_dataset/images/val/')
move_files_to_folder(test_images, '/content/Complete-Blood-Cell-Count-Dataset/new_dataset/images/test/')
move_files_to_folder(train_annotations, '/content/Complete-Blood-Cell-Count-Dataset/new_dataset/labels/train/')
move_files_to_folder(val_annotations, '/content/Complete-Blood-Cell-Count-Dataset/new_dataset/labels/val/')
move_files_to_folder(test_annotations, '/content/Complete-Blood-Cell-Count-Dataset/new_dataset/labels/test/')



In [ ]:
#create file contain paths for dataset and classes
with open('cbc_data.yaml', 'w') as f:
    f.write('train: ../Complete-Blood-Cell-Count-Dataset/new_dataset/images/train\n')
    f.write('val: ../Complete-Blood-Cell-Count-Dataset/new_dataset/images/val\n')
    f.write('test: ../Complete-Blood-Cell-Count-Dataset/new_dataset/images/test\n')
    f.write('nc: 3\n')
    f.write('names: ["RBC","WBC", "Platelets"]\n')


##Train 

In [ ]:
!python /content/yolov5/train.py  --img 416 --batch 32 --epochs 2 --data /content/cbc_data.yaml --weights yolov5s.pt


##Export 

In [ ]:
!python  /content/yolov5/export.py  --weights /content/yolov5/runs/train/exp/weights/best.pt --include tflite --img 416


##Predict 

In [ ]:
!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/exp/weights/best.torchscript  --img 640 --conf 0.25 --source /content/Complete-Blood-Cell-Count-Dataset/new_dataset/images/test/BloodImage_00347.jpg